<a href="https://colab.research.google.com/github/gmn-data-platform/gmn-data-endpoints/blob/main/gmn_data_analysis_template_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Development** Global Meteor Network Data Analysis Template

Starter template for GMN data analysis. Using [gmn-python-api](https://github.com/gmn-data-platform/gmn-python-api) (<a href="https://gmn-python-api.readthedocs.io/en/latest/">docs</a>) and [WesternMeteorPyLib](https://github.com/wmpg/WesternMeteorPyLib).

**Click on "Copy to Drive" above to allow changes to be saved.**

To use this **development** template:
1. Connect to a local Jupyter notebook server using the "Connect" dropdown in the top right following the instructions provided by Google Colab.
2. Install the target `gmn-python-api`, `wmpl` and `dtale` packages are installed in the Python 3.7 site-packages path.
3. Locally run the Docker services in gmn-data-endpoints (the value for `gmn_python_api.gmn_data_store_rest_api.QUERY_URL` should be replaced with `http://0.0.0.0:8001/gmn_data_store{table}.{data_format}?_shape={data_shape}`)
4. To run this, data up to 25/7/2019 should have been sucessful ingested into the gmn-data-store database.

---

## Load meteor/trajectory summary data into Pandas using gmn-python-api

In [ ]:
# Imports
from datetime import datetime
from gmn_python_api import get_daily_file_content_by_date
from gmn_python_api import read_meteor_summary_csv_as_dataframe
from gmn_python_api import get_meteor_summary_data_reader_compatible
import gmn_python_api.gmn_data_store_rest_api
gmn_python_api.gmn_data_store_rest_api.QUERY_URL = "http://0.0.0.0:8001/gmn_data_store{table}.{data_format}?_shape={data_shape}"

In [ ]:
import dtale
import dtale.app as dtale_app

In [ ]:
# REST API load test
meteor_summary_file_content_rest_api = get_meteor_summary_data_reader_compatible(where="beginning_utc_time__date=2019-07-24")[0]
meteor_summary_dataframe_rest_api = read_meteor_summary_csv_as_dataframe(meteor_summary_file_content_rest_api, csv_data_directory_format=False)
print(f"{meteor_summary_dataframe_rest_api['Vgeo (km/s)'].max()} km/s was the fastest geostationary velocity out of all meteors for that day.")
print(f"{meteor_summary_dataframe_rest_api.loc[meteor_summary_dataframe_rest_api['IAU (code)'] == 'PER'].shape[0]} meteors were estimated to be part of the Perseids shower.")
print(f"Station #{meteor_summary_dataframe_rest_api['Num (stat)'].mode().values[0]} recorded the highest number of meteors.")

In [ ]:
# Data directory load test
trajectory_summary_file_content_data_directory = get_daily_file_content_by_date(datetime(2019, 7, 24))
trajectory_summary_dataframe_data_directory = read_meteor_summary_csv_as_dataframe(trajectory_summary_file_content_data_directory, csv_data_directory_format=True)
print(f"{trajectory_summary_dataframe_data_directory['Vgeo (km/s)'].max()} km/s was the fastest geostationary velocity out of all meteors for that day.")
print(f"{trajectory_summary_dataframe_data_directory.loc[trajectory_summary_dataframe_data_directory['IAU (code)'] == 'PER'].shape[0]} meteors were estimated to be part of the Perseids shower.")
print(f"Station #{trajectory_summary_dataframe_data_directory['Num (stat)'].mode().values[0]} recorded the highest number of meteors.")

---

## WesternMeteorPyLib skyplot

Below is a skyplot (BEThel (deg) on y-axis, LAMgeo (deg) on x-axis) of meteors recorded on the 24th of July 2019. wmpl is used for rendering the plot. The red points indicate meteors estimated to be part of the Perseids shower.

In [ ]:
%matplotlib inline

import numpy as np
import wmpl
from wmpl.Utils.PlotCelestial import CelestialPlot
import matplotlib.pyplot as plt

ra = []
dec = []

ra.append(np.deg2rad((meteor_summary_dataframe_rest_api['LAMgeo (deg)'] - meteor_summary_dataframe_rest_api['Sol lon (deg)']) % 360))
dec.append(np.deg2rad(meteor_summary_dataframe_rest_api['BEThel (deg)']))
colors = ["red" if str(x) == "PER" else "white" for x in meteor_summary_dataframe_rest_api['IAU (code)']]

plt.rcParams['figure.figsize'] = 15, 25
celes_plot = CelestialPlot(ra, dec, projection='sinu', lon_0=-90)
celes_plot.scatter(ra, dec, c=colors, s=15)

---

## D-Tale
**Currently not working for dev template due to https://github.com/man-group/dtale/issues/661**

[D-Tale](https://github.com/man-group/dtale) is a visualiser for Pandas data structures. It can be used to explore and create plots for `meteor_summary_dataframe_rest_api`. Run the cells below and click on the link generated to access D-Tale.

In [ ]:
!pip install -U dtale

import pandas as pd

import dtale
import dtale.app as dtale_app

dtale_app.USE_COLAB = True
# dtale_app.USE_NGROK = False

dtale.show(meteor_summary_dataframe_rest_api)